In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA



In [2]:
## Read the ppdfs from the folder
loader=PyPDFDirectoryLoader("./us_census")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3', metadata={'source': 'us_census\\acsbr-015.pdf', 'page': 0})

In [5]:
len(final_documents)

316

In [3]:
## Embedding Using Huggingface
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

c:\DL projects\Langchaing\Updated-Langchain\lang_env\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\DL projects\Langchaing\Updated-Langchain\lang_env\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mmanoj\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Develope

In [5]:
em=huggingface_embeddings.embed_query("King and Queen")

In [9]:
np.array(em)

array([-1.82168838e-02,  9.99695659e-02,  8.10765028e-02, -3.27630038e-03,
        7.98593275e-03,  1.55359851e-02,  1.74987037e-02, -3.22268382e-02,
        6.07669428e-02, -1.84381567e-02, -2.51246616e-02,  1.98375341e-03,
        3.61564569e-02, -2.01915484e-02,  2.51618922e-02, -3.14232963e-03,
       -2.94486694e-02, -2.13192720e-02, -9.73709971e-02,  6.49688095e-02,
        3.85748856e-02, -4.43241596e-02, -3.61262225e-02,  1.80379339e-02,
       -1.60644855e-02, -7.69400364e-03,  7.30349310e-03,  1.80179179e-02,
       -2.43312493e-02, -1.45794302e-01, -1.05483206e-02, -2.03213599e-02,
        1.67371240e-02, -1.31763658e-02, -4.02104408e-02, -1.73749086e-02,
       -1.32598653e-02,  7.33525082e-02,  1.52545758e-02, -2.52583511e-02,
        2.53770929e-02, -4.54790592e-02,  1.48559930e-02,  1.35691948e-02,
        8.21128895e-04,  6.98513910e-02,  1.52171799e-03,  8.17486458e-03,
       -1.20595023e-02,  1.68998670e-02, -2.87922341e-02,  2.68402928e-03,
       -9.67929792e-03, -

In [4]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-8.46568272e-02 -1.19099068e-02 -3.37892212e-02  2.94559114e-02
  5.19160256e-02  5.73840030e-02 -4.10018116e-02  2.74267606e-02
 -1.05128206e-01 -1.58055872e-02  7.94858783e-02  5.64318895e-02
 -1.31765716e-02 -3.41544449e-02  5.81600517e-03  4.72547635e-02
 -1.30746737e-02  3.12989787e-03 -3.44226249e-02  3.08406334e-02
 -4.09086272e-02  3.52737717e-02 -2.43761223e-02 -4.35830802e-02
  2.41503119e-02  1.31986486e-02 -4.84455097e-03  1.92347057e-02
 -5.43912649e-02 -1.42735094e-01  5.15529374e-03  2.93115713e-02
 -5.60810752e-02 -8.53529759e-03  3.14141288e-02  2.76736636e-02
 -2.06188671e-02  8.24231505e-02  4.15425412e-02  5.79654835e-02
 -3.71586867e-02  6.26162579e-03 -2.41390318e-02 -5.61797759e-03
 -2.51715723e-02  5.04976278e-03 -2.52801497e-02 -2.91943387e-03
 -8.24041571e-03 -5.69603927e-02  2.30822600e-02 -5.54215023e-03
  5.11555411e-02  6.09937273e-02  6.49766028e-02 -5.38513996e-02
  2.19109710e-02 -2.54194979e-02 -4.49223891e-02  4.22458909e-02
  4.75252643e-02  7.23222

In [10]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:10],huggingface_embeddings)

In [11]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [12]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002A2C9768AF0> search_kwargs={'k': 3}


In [16]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']=""

The Hugging Face Hub is an platform with over 350k models, 75k datasets, and 150k demo apps (Spaces), all open source and publicly available, in an online platform where people can easily collaborate and build ML together.

In [17]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the health insurance coverage?"
hf.invoke(query)

e:\Langchain\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


'What is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insurer and the insured. The insurer agrees to pay the insured for the medical expenses incurred by the insured. The insured agrees to pay the premium to the insurer.\n\nWhat is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insurer and the insured. The insurer agrees to pay the insured for the medical expenses incurred by the insured.'

In [ ]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf 
llm.invoke(query)

In [22]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [24]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [25]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [28]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [29]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])


Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

comparison of ACS and CPS ASEC measures 
of health insurance coverage, refer to < www.
census.gov/topics/health/health-insurance/
guidance.html >.
9 Respondents may have more than one 
health insurance coverage type at the time 
of interview. As a result, adding the total 
number of people with private coverage and 
the total number with public coverage will 
sum to more than the total number with any 
coverage.• From 2021 to 2022, nine states 
reported increases in private 
coverage, while seven reported 
decreases (Appendix Table B-2). 
DIFFERENCES IN THE 
UNINSURED RATE BY STATE 
IN 2022
In 2022, uninsured rates at the 
time of interview ranged across 
states from a low of 2.4 percent 
in Massachusetts to a high of 16.6 
percent in Texas, compared to the 
national rate of 8.0 percent.10 Ten 
of the 15 states with uninsured 
10 The uninsured rates in the Distr